<a href="https://colab.research.google.com/github/jongjunkim/Transaction_History_DAG/blob/main/Transaction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

In [2]:
!wget https://jongjun.s3.us-east-2.amazonaws.com/transaction_history.csv

--2023-11-15 19:28:45--  https://jongjun.s3.us-east-2.amazonaws.com/transaction_history.csv
Resolving jongjun.s3.us-east-2.amazonaws.com (jongjun.s3.us-east-2.amazonaws.com)... 52.219.109.138, 3.5.130.120, 3.5.131.121, ...
Connecting to jongjun.s3.us-east-2.amazonaws.com (jongjun.s3.us-east-2.amazonaws.com)|52.219.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44491 (43K) [binary/octet-stream]
Saving to: ‘transaction_history.csv’

transaction_history 100%[===================>]  43.45K  --.-KB/s    in 0.06s   

2023-11-15 19:28:45 (703 KB/s) - ‘transaction_history.csv’ saved [44491/44491]



In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #1")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [5]:
df = spark.read.format("text").load("transaction_history.csv")

In [6]:
df.show()

+--------------------+
|               value|
+--------------------+
|1,2023-11-12,VENM...|
|2,2023-11-12,Chec...|
|3,2023-11-12,Tran...|
|4,2023-11-09,Chec...|
|5,2023-11-09,Jong...|
|6,2023-11-08,Tran...|
|7,2023-11-08,Elec...|
|8,2023-11-07,Tran...|
|9,2023-11-07,Elec...|
|10,2023-11-04,Tra...|
|11,2023-11-03,Che...|
|12,2023-11-03,Che...|
|13,2023-11-02,Ube...|
|14,2023-11-02,Ube...|
|15,2023-11-02,Chi...|
|16,2023-11-01,Che...|
|17,2023-10-30,Jon...|
|18,2023-10-27,Ins...|
|19,2023-10-27,Tra...|
|20,2023-10-26,Che...|
+--------------------+
only showing top 20 rows



In [7]:
df = spark.read.format("csv")\
      .load("transaction_history.csv")\
      .toDF("id", "Date", "description", "debit", "credit", "classification" )

In [8]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- debit: string (nullable = true)
 |-- credit: string (nullable = true)
 |-- classification: string (nullable = true)



In [26]:
#Set Structure
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
from pyspark.sql.types import StructType, StructField

schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("Date", DateType(), True), \
                     StructField("description", StringType(), True), \
                     StructField("debit", FloatType(), True), \
                     StructField("credit", FloatType(), True), \
                     StructField("classification", StringType(), True)])

In [28]:
df = spark.read.schema(schema).csv("transaction_history.csv")

In [29]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- description: string (nullable = true)
 |-- debit: float (nullable = true)
 |-- credit: float (nullable = true)
 |-- classification: string (nullable = true)



In [30]:
df.count()

841

In [31]:
df.show()

+---+----------+--------------------+------+-------+-----------------+
| id|      Date|         description| debit| credit|   classification|
+---+----------+--------------------+------+-------+-----------------+
|  1|2023-11-12|VENMO  *STARBUCK ...|  25.0|    0.0|             null|
|  2|2023-11-12|Check Insomnia Co...| 11.72|    0.0|Food &amp; Dining|
|  3|2023-11-12|   Transfer to Venmo|427.71|    0.0|         Transfer|
|  4|2023-11-09|Check Binghamton ...|  1.38|    0.0|    Uncategorized|
|  5|2023-11-09|         Jongjun Kim|   0.0| 454.45|         Paycheck|
|  6|2023-11-08| Transfer from Venmo|   0.0|   35.0|         Transfer|
|  7|2023-11-08|        Electric Gas| 86.73|    0.0|        Utilities|
|  8|2023-11-07|   Transfer to Venmo| 68.21|    0.0|         Transfer|
|  9|2023-11-07|        Electric Gas| 86.73|    0.0|        Utilities|
| 10|2023-11-04|   Transfer to Venmo|  15.0|    0.0|         Transfer|
| 11|2023-11-03|      Check 30518793| 750.0|    0.0|            Check|
| 12|2

#Spark SQL

In [32]:
df.createOrReplaceTempView("transaction_history")

In [49]:
query1 = spark.sql("""SELECT Date, description, debit, classification
FROM transaction_history
where debit = (SELECT Max(debit) from transaction_history)
""")

In [50]:
query1.show()

+----------+-----------+------+--------------+
|      Date|description| debit|classification|
+----------+-----------+------+--------------+
|2022-06-09| Withdrawal|1790.0| Uncategorized|
+----------+-----------+------+--------------+



In [87]:
#ranking over debit
query2 = spark.sql("""
Select Date, description, debit, credit, classification from
(SELECT Date, description, debit, credit, classification, row_number() over(partition by classification order by debit desc) as rn
FROM transaction_history) as sub
where sub.rn = 1
order by debit desc
""")

In [88]:
query2.show()

+----------+--------------------+------+------+--------------------+
|      Date|         description| debit|credit|      classification|
+----------+--------------------+------+------+--------------------+
|2022-06-09|          Withdrawal|1790.0|   0.0|       Uncategorized|
|2023-08-18|      Check 30502923|1475.0|   0.0|               Check|
|2022-05-05|   Check Suny Canton| 942.5|   0.0|           Education|
|2022-11-01|   Transfer to Venmo| 746.0|   0.0|            Transfer|
|2022-06-05|      ATM Withdrawal| 500.0|   0.0|                Cash|
|2022-07-05|   American Airlines| 400.6|   0.0|          Air Travel|
|2023-03-13|              Airbnb|187.15|   0.0|              Travel|
|2022-11-27|                Zara| 184.0|   0.0|            Shopping|
|2022-07-10|Check Home Scienc...|127.29|   0.0|                Home|
|2023-09-18|            Spectrum|107.73|   0.0|Bills &amp; Utili...|
|2022-01-31|           Instacart|106.92|   0.0|           Groceries|
|2023-11-08|        Electric Gas| 

In [75]:
driver_results = query2.collect()
for r in driver_results:
    print(r)

Row(Date=datetime.date(2023, 11, 12), description='VENMO  *STARBUCK 5814034538 1112114538 331618594702                    11 739.38 11/15/2023 12:00:00 AM', debit=25.0, credit=0.0, classification=None)
Row(Date=datetime.date(2022, 7, 5), description='American Airlines', debit=400.6000061035156, credit=0.0, classification='Air Travel')
Row(Date=datetime.date(2023, 2, 12), description='Check Parkway Wine &amp; Spirits', debit=44.2599983215332, credit=0.0, classification='Alcohol &amp; Bars')
Row(Date=datetime.date(2019, 12, 27), description='At ATM Broome County Transit Aut Chenango Stbinghamton', debit=23.0, credit=0.0, classification='Auto &amp; Transport')
Row(Date=datetime.date(2023, 9, 18), description='Spectrum', debit=107.7300033569336, credit=0.0, classification='Bills &amp; Utilities')
Row(Date=datetime.date(2022, 1, 27), description='Check Zybook College Txtbook', debit=77.0, credit=0.0, classification='Business Services')
Row(Date=datetime.date(2022, 6, 5), description='ATM Wi

In [102]:
#Consumption Patterns
query3 = spark.sql("""
Select classification, Round(sum(debit),2) as Total_debit,
Concat(Round(sum(debit) / (select sum(debit) from transaction_history) * 100,2), "%") as Percentage
FROM transaction_history
where classification is not null
group by classification
order by sum(debit) desc limit 20
""")

In [103]:
query3.show()

+--------------------+-----------+----------+
|      classification|Total_debit|Percentage|
+--------------------+-----------+----------+
|            Transfer|   10230.86|    29.08%|
|               Check|     5153.0|    14.65%|
|       Uncategorized|    4155.94|    11.81%|
|                Cash|     3970.0|    11.29%|
|           Groceries|    2960.24|     8.41%|
|            Shopping|    1679.01|     4.77%|
|           Education|    1330.47|     3.78%|
|           Fast Food|     1032.2|     2.93%|
|              Travel|     878.75|      2.5%|
|Bills &amp; Utili...|      733.8|     2.09%|
|        Coffee Shops|     467.45|     1.33%|
|   Food &amp; Dining|     438.08|     1.25%|
|          Air Travel|      400.6|     1.14%|
|Rental Car &amp; ...|     375.59|     1.07%|
|           Utilities|     241.76|     0.69%|
|                Home|     232.83|     0.66%|
|         Restaurants|     142.53|     0.41%|
|  Alcohol &amp; Bars|     134.21|     0.38%|
|   Business Services|       98.6|